In [1]:
import openai
import json
import itertools

In [2]:
with open('openai_key.txt', encoding='utf-8') as f:
    openai_key = f.readlines()[0]

In [3]:
openai.api_key = openai_key

In [4]:
def chat(conversation, prompt ,model='gpt-3.5-turbo'):

  conversation.append(prompt)

  if model == 'gpt-3.5-turbo':
    response = openai.ChatCompletion.create(
      model = 'gpt-3.5-turbo',
      messages = conversation,
      temperature = 0.7)
    
    if response.choices:
      return response.choices[0].message.content.strip()
    else:
      return None
  
  # if model == 'text-davinci-003':
  #   response = openai.Completion.create(
  #     engine='text-davinci-003', 
  #     prompt=prompt)
    
  #   if response.choices:
  #     return response.choices[0].text.strip()
  #   else:
  #     return None


In [5]:
with open('initial_prompts.json') as f:
    initial_prompts = json.load(f)

In [6]:
prompt = {"role": "user", "content": "Annie must like Starbucks because all white girls like Starbucks."}

In [7]:
chat(initial_prompts[:9], prompt)

'Title: "Stereotyping and Individual Preferences"\nFact: Stereotyping involves making assumptions about a person or group based on limited information or characteristics that are not necessarily indicative of their individual preferences or experiences. It is important to avoid stereotyping and to recognize the diversity of individual preferences and experiences.\nMyth: Some people believe that all white girls like Starbucks, based on a stereotype.\nFallacy: This myth is based on the logical fallacy of stereotyping, as it makes assumptions about an entire group of people based on a limited characteristic. In reality, individual preferences vary widely and cannot be reduced to simplistic stereotypes.\nFact: Individual preferences cannot be determined by stereotypes based on limited characteristics such as race or gender. Instead, it is important to recognize and respect the diversity of individual experiences and preferences. Let\'s not spread misinformation by relying on stereotypes.'